## Part_3

### 0. Import Libraries.

In [369]:
import numpy as np
import pandas as pd

import yfinance as yf
import finta


import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

import gc; gc.enable()

from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split,cross_validate

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer

from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report


from scipy.signal import argrelextrema
from scipy.signal import savgol_filter

from sklearn.model_selection import StratifiedKFold

from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from category_encoders import WOEEncoder, TargetEncoder
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")


### 1. Define Helper Functions.

In [371]:
def smooth(series):
    '''
    This function smoothes target/close price. 
    '''
    series = savgol_filter(series, 21, 2)
    series = savgol_filter(series, 11, 2)
    series = savgol_filter(series, 11, 1)
    return series
    
def loc_minima(series, order=10):
    '''
    Function returns pandas series with values 1&0: if local minima = 1, otherwise = 0.
    Where,
    series: e.g. = df.close
    order: number of days(both sides) of the local minimum point.
    
    '''
    # get the indeces of local minima with specified order(number of days from local min)
    local_min = argrelextrema(series.values, np.less_equal, order=order)[0].tolist()
    
    # get pandas series with 0s.
    min_series = 0*series
    
    # update series by assigning 1s to local minima(using list of indeces: local_min) 
    min_series[local_min] = 1
    
    gc.collect()
    return min_series

def loc_maxima(series, order=10):
    '''
    Function returns pandas series with values 1&0: if local maxima = 1, otherwise = 0.
    Where,
    series: e.g. = df.close
    order: number of days(both sides) of the local maximum point.
    
    '''
    
    # get the indeces of local maxima with specified order(number of days from local max)
    local_max = argrelextrema(series.values, np.greater_equal, order=order)[0].tolist()
    
    # get pandas series with 0s.
    max_series = 0*series
    
    # update series by assigning 1s to local maxima(using list of indeces: local_max) 
    max_series[local_max] = 1

    gc.collect()
    return max_series

def loc_extrema(series, order=10):
    
    '''
    Function returns pandas series with values 0,1,-1: 
    if local minima = 1, if local maxima = -1, otherwise = 0.
    
    Where,
    series: e.g. = df.close
    order: number of days(both sides) of the local extremum point.
    
    '''
    loc_max = loc_maxima(series,order)
    loc_min = loc_minima(series,order)
    return loc_min - loc_max

def calc_slope(x):
    '''
    Function calculates 'slope' of x
    '''
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope

def calc_acc(x):
    '''
    Function calculates 'acceleration' of x
    '''
    acc = np.polyfit(range(len(x)), x, 2)[0]
    return acc


def col_rename(data):
    '''
    Function renames columns.
    '''
    data.dropna(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.sort_values(by='Date',inplace=True)
    data.set_index('Date',inplace=True)
    data.index.rename('date',inplace=True)
    data.rename(str.lower, axis='columns',inplace=True)
    data.rename(columns={"stock splits": "stock_splits"},inplace=True)
    #data.columns = ['close', 'dividends', 'high', 'low','open','stock_splits','ticker','volume']
    return data

def split_check(data):
    '''
    No splits returns True
    if splits return False
    '''
    if len(data.stock_splits.unique())==1 and data.stock_splits.unique()[0]==False:
        return True
    else:
        return False

def reduce_df(data):
    '''
    Returns reduced data frame.
    '''
    data = col_rename(data) # renames columns
    
    try:
        data = data[['ticker','open','high','low','close','volume']]
        return data
    except:
        print('Error occured!')
    
    
def display(data,x=5,y=5):
    '''
    Function shows head and tail of data frame.
    '''
    return data.head(x).append(data.tail(y))

def plot(data, size = (18,6),title='Plot',legend=True):
    '''
    plots dataFrame or dataSeries
    '''
    #plt.figure(figsize=size)
    data.plot(title=title,legend=legend,figsize=size)
    plt.show()
    
def plot_df(df, columns, size = (18,8), scale = 10*[1]):
    
    '''
    Plots dataFrame or dataSeries.
    '''

    #plt.style.use('fivethirtyeight')
    plt.style.use('seaborn-darkgrid')

    # create a color palette
    palette = plt.get_cmap('tab10')

    plt.figure(figsize=size)

    #multiple line plot
    #plot main line
    plt.plot(df.index, scale[0]*df[columns[0]], marker='', color='blue', linewidth=3,
             alpha=1, label = columns[0], linestyle='solid')
    
    #plot secondary lines
    num=0
    for col in df[columns].drop(columns[0], axis=1):
        num+=1
        plt.plot(df.index, scale[num]*df[col], marker='', color=palette(num), linewidth=1, 
                 alpha=0.7, label=col, linestyle='solid') #markersize=1


    # Add legend
    plt.legend()

    # Change xlim
    #plt.xlim(0,12)

    # Add titles
    plt.title("Plot", fontsize=20, fontweight=3, color='black')
    plt.xlabel("Date",fontsize=14, fontweight=3, color='black')
    plt.ylabel("Close Price",fontsize=14, fontweight=3, color='black')
    plt.show()

    
def add_features(df):
    
    '''
    Function returns data frame with features.
    '''
       
    ##### Prepare columns for finta module https://pypi.org/project/finta/
    ohlc = ['open', 'high', 'low', 'close']
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    
    df_accum = pd.DataFrame()

    
    ##### Loop through
    for ticker in df['ticker'].unique():
        
        df_tmp = df.loc[df["ticker"]==ticker]

        ##### Get close price change for 1-15, 20, 25, 30 days ago

        # Percentage change between the current and a prior closing price
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html

        # Compare percentage change of today's price refernce to price of'lag' day ago
        lags = list(range(1,16))+[20,25,30]
        for i in lags:
            df_tmp[f'pct_lag_{i}']=df_tmp['close'].pct_change(periods=i)



        ##### Get the rolling mean and std deviation of pct_change

        # Compare percentage change of today's price refernce to price'lag' day ago
        lags = [1,5,10,15,10] 

        # Rolling windows = 5,10,15,20 days
        windows = [5,10,15,20] 
        for i in lags:
            for window in windows:
                df_tmp[f'pct_r_mean_{i}_{window}'] = df_tmp[f'pct_lag_{i}'].rolling(window).mean()
                df_tmp[f'pct_r_std_{i}_{window}'] = df_tmp[f'pct_lag_{i}'].rolling(window).std()


        ##### Get the exponential mean and std deviation of pct_change
        # finta.TA.EMA(df[ohlc],period=5) is the same df.close.ewm(span = 5).mean()

        # use pct_lag_1 here
        # use span = 5 and 10 periods or alpha = 2/(span+1)  (period is not the same as day)        
        df_tmp['pct_exp_mean_1_5'] = df_tmp['pct_lag_1'].ewm(span=5).mean()
        df_tmp['pct_exp_std_1_5'] = df_tmp['pct_lag_1'].ewm(span=5).std()

        df_tmp['pct_exp_mean_1_10'] = df_tmp['pct_lag_1'].ewm(span=10).mean()
        df_tmp['pct_exp_std_1_10'] = df_tmp['pct_lag_1'].ewm(span=10).std()


        ##### Get 'Direction'='MACD' - 'Signal'

        # MACD= 12_Period EMA − 26_Period EMA
        # https://www.investopedia.com/terms/m/macd.asp
        # df.close.ewm(span=12).mean()-df.close.ewm(span=26).mean() is the same as:
        # finta.TA.MACD(df[ohlc]), also returns 9 day signal line

        df_tmp['macd'] = (df_tmp.close.ewm(span=12).mean()-df_tmp.close.ewm(span=26).mean())

        # signal line is 9 period EMA of the MACD line
        df_tmp['signal']= df_tmp['macd'].ewm(span=9).mean()

        # diffrence between MACD and Signal line indicates bullish
        # or bearish direction of the market
        df_tmp['direction'] = df_tmp['macd']-df_tmp['signal']


        ##### Get 'slope' and 'acceleration'                          
        df_tmp['slope'] = df_tmp.close.rolling(9).apply(calc_slope,raw=False)
        df_tmp['acc'] = df_tmp.close.rolling(11).apply(calc_acc,raw=False)


        ### Get parabolic sar: 'stop and reverse'
        # https://www.investopedia.com/trading/introduction-to-parabolic-sar/
        # e.g value of af = 0.01  could keep in position longer time 
        df_tmp['sar'] = finta.TA.SAR(df_tmp[ohlc].copy(), af=0.025, amax=0.15)# not use as feature, plot only  

        # get normalized sar as signal: 
        #if 0<norm<1 then uptrend, if norm < 0, downtrend
        # here is an imballance possible, however as we do two separate models, 
        # for min and max target, may need to adjust
        df_tmp['sar_norm'] = 1- df_tmp.sar/df_tmp.close

        # do the same for moving avg 50,100,150,200
        windows=[50,100,150,200]

        # get normalized rolling_mean as signal: if 0<norm<1 then uptrend, if norm < 0, downtrend
        # here is an imballance possible, however as we do two separate models, 
        # for min and max target
        for win in windows: 
            df_tmp[f'r_mean_{win}'] = df_tmp.close.rolling(win).mean() # not use as feature, plot only
            df_tmp[f'r_mean_{win}_norm'] = 1 - df_tmp[f'r_mean_{win}']/df_tmp.close

        # macd: if short moving avg > long moving avg 
        # normalize: divide by short moving avg, and if 0<norm<1 then uptrend, if norm < 0, downtrend

        df_tmp[f'macd_50_100_norm'] = 1 - df_tmp[f'r_mean_100']/df_tmp[f'r_mean_50']
        df_tmp[f'macd_50_150_norm'] = 1 - df_tmp[f'r_mean_150']/df_tmp[f'r_mean_50']
        df_tmp[f'macd_50_200_norm'] = 1 - df_tmp[f'r_mean_200']/df_tmp[f'r_mean_50']
        df_tmp[f'macd_100_200_norm'] = 1 - df_tmp[f'r_mean_200']/df_tmp[f'r_mean_100']


        ### Get ADX: Average Directional Index
        # The A.D.X. is 100 * smoothed moving average of absolute value (DMI +/-) 
        # divided by (DMI+ + DMI-). ADX does not indicate trend direction or momentum,
        # only trend strength.
        # Here I normalised by dividing 30(assumed here above 30 is moderately strong trend).
        # https://www.investopedia.com/terms/a/adx.asp
        df_tmp['adx'] = (finta.TA.ADX(df_tmp[ohlc].copy(), period=14))/30


        ### Get RSI relative strength index is a momentum indicator
        # https://www.investopedia.com/terms/r/rsi.asp
        df_tmp['rsi'] = finta.TA.RSI(df_tmp[ohlc].copy(), period=14)
        
#         df_tmp['vwap'] = finta.TA.VWAP(df_tmp[ohlcv].copy())
        
        ### add_targets
        # get local minima and maxima (10 days both side from local extrema)
        df_tmp['target_min'] = loc_minima(df_tmp.close,order=10)
        df_tmp['target_max'] = loc_maxima(df_tmp.close,order=10)

        # get local extrema, its the same as above, just for plotting purpose
        # local minima(=1) and maxima(=-1)
        df_tmp['extrema'] = loc_extrema(df_tmp.close,order=10)

        #df_tmp.dropna(inplace=True)
        
        df_accum = pd.concat([df_accum,df_tmp],sort='False')

    df_accum.dropna(inplace=True)
    df_accum.sort_values(by='date',inplace=True)
    gc.collect()
    
    return df_accum


def roi_calc(data):
    
    '''
    Function calculates ROI for each stock ticker.
    Returns two dataframes: 
    - ROI for each stock for all test period.
    - DF with positions in and out.
    '''
    
    data = data[['ticker','open','high','low','close','predicted']]
    data['pos_in']= data['predicted']
    data['pos_out'] = 0
    data['roi']=1
    data.fillna(0,inplace=True)


    t_stop = -0.045 #### trail stop loss 4.5%
    h_stop = 0.95 # hard stop loss 5% per initial buy

    # create empty data frame, will use for assemble back all stock tickers
    concat_all_df = pd.DataFrame()

    #loop thru each ticker as df-s
    for tick in data['ticker'].unique():

        # create data frame for each stock ticker
        ticker_df = data.loc[data['ticker']==tick]

        # reset indeces
        ticker_df.reset_index(inplace=True)

        # index of the first position_in=1
        first_idx = ticker_df.loc[ticker_df['pos_in']==1].index[0]

        # start df with first in_position = 1
        ticker_df = ticker_df[first_idx:]

        # fix indeces
        ticker_df.reset_index(inplace=True)
        del ticker_df['index']

        # get the list of  all position_in global indeces for one ticker,
        # e.g., for 'BMY'.
        idx_pos_in = ticker_df.loc[ticker_df.pos_in ==1].index.tolist()

        # add the last index to the list if it is not in_postion
        if (len(ticker_df)-1) not in idx_pos_in:
            idx_pos_in.append(len(ticker_df)-1)

        # create empty data frame, will use for assemble back
        concat_df = pd.DataFrame()

        # loop thru all indeces of the list( which is indeces of postion_in) 
        # for one ticker/symbol, e.g. 'BMY'
        for i in range(len(idx_pos_in)):

            # to avoid out of range for the last section of position_in 
            # till the end of the data frame 
            if i < len(idx_pos_in)-1:

                # start slicing the data frame of one ticker for multiple dataframes, 
                # based on the position_in, end - start is delta 
                # between one postion_in and next position_in
                start = idx_pos_in[i] # global index of position_in
                end = idx_pos_in[i+1] # global index of next position_in

                # create a copy of a sub data frame(from one position_in till next position_in )
                tmp = ticker_df.loc[start:end].copy() 

                # create trailing loss column
                tmp['pct_loss'] = tmp['close'].pct_change()

                # fill NaN (created after pct_change) with 0
                tmp.fillna(0,inplace=True)

                # loop inside this sub data frame by indeces (j = index)
                for j in range(start, end+1):

                    # hard loss today(today means at global index j)
                    h_loss = tmp['close'][j]/tmp['close'][start]
                    # trail loss today
                    t_loss = tmp['pct_loss'][j] 

                    # define if need to stop; 
                    # e.g., h_loss = 0.95 or t_loss = -0.07 reqire to stop
                    # as it excced limit specified above as h_stop and t_stopp
                    if h_loss <= h_stop or t_loss <= t_stop:

                        # assign to postion_out 1 as we exceeded allowed stop loss
                        tmp.loc[j,'pos_out']=1

                        # assign to ROI current h_loss, as it is equal to ROI
                        tmp.loc[j,'roi'] = h_loss

                        # break for 'j' loop
                        break

                    # if we didnt exceed any loss stops, than last 
                    # position_out = position_in=1
                    # than means we exit postion and enter it again with the same closed price
                    # its 'fake' position close
                    if j==(end):
                        tmp.loc[j,'pos_out']=1
                        tmp.loc[j,'roi'] = h_loss

                # remove first row of the sub data frame, starting from the second
                if i>0:
                    tmp = tmp[1:]
                concat_df = pd.concat([concat_df,tmp],join='outer')

        concat_df['roi_cumprod']=concat_df['roi'].cumprod()
        
        # dataframe with all data, including roi for each stock
        concat_all_df = pd.concat([concat_all_df,concat_df],join='outer')
    
    del concat_df, tmp
    gc.collect()
    
    roi_all_df = concat_all_df.groupby('ticker')['roi_cumprod'].agg(['last'])
    roi_all_df.columns=['roi']
    
    return roi_all_df, concat_all_df

def calc_sar(data):
    '''
    This function returns ROI using SAR for each stock-ticker.
    '''
    ohlc= ['open', 'high', 'low', 'close']
    data = data[ohlc+['ticker']]


    # create empty data frame, will use for assemble back all stock tickers
    all_sar_data= pd.DataFrame()

    #loop thru each ticker as df-s
    for tick in data['ticker'].unique():
        # create data frame for each stock ticker
        ticker_df = data.loc[data['ticker']==tick]

        # typically af=0.01 for stocks and af=0.02 for currencies (forex)
        # but pennies and small-caps behaviour is closer to currencies than stocks
        # https://www.investopedia.com/trading/introduction-to-parabolic-sar/

        ticker_df['sar'] = finta.TA.SAR(ticker_df[ohlc], af=0.025, amax=0.15) 
        # e.g value of af = 0.01  could keep in position longer time 

        ticker_df['position'] = (ticker_df.sar < ticker_df.close).astype(np.int)

        #https://www.investopedia.com/terms/a/adx.asp
        ticker_df['adx'] = (finta.TA.ADX(ticker_df[ohlc], 14))/30

        ticker_df['region'] = (ticker_df.position != ticker_df.position.shift()).cumsum()

        stats = ticker_df.loc[ticker_df.position == 1].groupby(['ticker','region'])[['close']].agg(['first', 'last', 'count'])
        stats.columns = ['opened', 'closed', 'days']

        # Return on investment
        stats['roi'] = stats.closed / stats.opened 

        # Profits & Losses
        stats['pnl'] = stats.roi - 1.0

        # Profits at a fixed amount - trading without reinvestment
        stats['pnl_cumulat'] = stats['pnl'].cumsum()

        # Profits at 100% portfolio value - trading with 100% reinvestment
        stats['roi_compound'] = stats['roi'].cumprod()
        stats = stats[-1:]


        # dataframe with all data, including roi for each stock
        all_sar_data = pd.concat([all_sar_data,stats],join='outer')

        del ticker_df, stats
        gc.collect()
    all_sar_data.reset_index(inplace=True)
    
    return all_sar_data

### 2. Get and prepare data.
### 2.1. Get from stock_500_full.csv.

In [15]:
df = pd.read_csv('stocks_500_full.csv')
df = reduce_df(df)
df

,ticker,open,high,low,close,volume
date,,,,,,
2010-05-19,BMY,16.99,17.27,16.88,17.17,16424000.0
2010-05-19,OHI,9.28,9.48,9.07,9.38,1532600.0
2010-05-19,EVRG,15.68,15.73,15.36,15.47,1529300.0
2010-05-19,GRMN,22.04,23.29,21.97,22.80,2383000.0
2010-05-19,ALE,24.88,25.14,24.53,24.79,154800.0
...,...,...,...,...,...,...
2020-05-18,EXC,36.32,37.84,36.02,37.54,6307400.0
2020-05-18,VTR,29.18,32.57,29.13,32.26,5177100.0
2020-05-18,NTES,385.00,385.98,374.01,381.88,656136.0


In [16]:
# leave only stock tickers which have data from 2010 and till 2020-05-18 
start_list = df['2010'].ticker.unique().tolist()
df = df.loc[df.ticker.isin(start_list)]

end_list = df['2020-05-18':].ticker.unique().tolist()
df = df.loc[df.ticker.isin(end_list)]

In [18]:
df.ticker.nunique()

409

In [19]:
df

,ticker,open,high,low,close,volume
date,,,,,,
2010-05-19,BMY,16.99,17.27,16.88,17.17,16424000.0
2010-05-19,OHI,9.28,9.48,9.07,9.38,1532600.0
2010-05-19,EVRG,15.68,15.73,15.36,15.47,1529300.0
2010-05-19,GRMN,22.04,23.29,21.97,22.80,2383000.0
2010-05-19,ALE,24.88,25.14,24.53,24.79,154800.0
...,...,...,...,...,...,...
2020-05-18,VZ,55.65,56.20,55.04,55.72,12197445.0
2020-05-18,EXC,36.32,37.84,36.02,37.54,6307400.0
2020-05-18,VTR,29.18,32.57,29.13,32.26,5177100.0


In [21]:
df.describe()

,open,high,low,close,volume
count,1.029067e+06,1.029067e+06,1.029067e+06,1.029067e+06,1.029067e+06
mean,6.727025e+01,6.800579e+01,6.652790e+01,6.729009e+01,3.365059e+06
std,1.507440e+02,1.524795e+02,1.490433e+02,1.508123e+02,8.805755e+06
min,6.400000e-01,6.700000e-01,6.000000e-01,6.300000e-01,0.000000e+00
25%,1.950000e+01,1.973000e+01,1.927000e+01,1.950000e+01,3.502000e+05
50%,3.627000e+01,3.665000e+01,3.589000e+01,3.628000e+01,1.055900e+06
75%,6.608000e+01,6.675000e+01,6.541000e+01,6.610000e+01,2.918500e+06
max,4.025000e+03,4.071130e+03,3.986950e+03,4.037770e+03,6.166205e+08


### 3. Add features to data frame.

In [23]:
df = add_features(df)
df

,acc,adx,close,direction,extrema,high,low,macd,macd_100_200_norm,macd_50_100_norm,...,r_mean_50_norm,rsi,sar,sar_norm,signal,slope,target_max,target_min,ticker,volume
date,,,,,,,,,,,,,,,,,,,,,
2011-03-03,0.013042,0.673802,19.64,0.039419,0.0,19.70,19.15,-0.011716,0.018264,-0.006424,...,0.014012,65.315130,18.970000,0.034114,-0.051135,0.047167,0.0,0.0,BMY,31795200.0
2011-03-03,0.048368,1.161528,36.16,-0.135454,0.0,36.47,34.94,0.427091,0.061482,0.041612,...,0.065127,61.072596,33.440000,0.075221,0.562544,-0.102667,0.0,0.0,WTS,786600.0
2011-03-03,0.003928,1.651108,31.71,0.102823,0.0,31.79,31.29,0.413437,0.020550,0.009856,...,0.052942,79.401427,30.944126,0.024152,0.310614,0.141667,0.0,0.0,D,2179900.0
2011-03-03,0.040093,0.957126,32.90,0.313865,-1.0,33.01,32.21,-0.209712,0.090992,-0.046692,...,0.007982,72.471100,30.131000,0.084164,-0.523577,0.219500,1.0,0.0,RDY,285400.0
2011-03-03,0.019277,0.933164,26.97,-0.036144,0.0,27.02,26.60,0.071601,0.022988,0.005639,...,0.030056,57.798985,25.442500,0.056637,0.107745,0.065167,0.0,0.0,LSI,92600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-18,0.032401,1.042237,24.13,0.149860,0.0,24.26,21.46,-2.701234,-0.318931,-0.873178,...,-0.193875,52.150196,27.231995,-0.128553,-2.851094,-0.285167,0.0,0.0,UAL,72649600.0
2020-05-18,0.066725,0.972561,71.32,-0.317740,0.0,73.07,70.70,-2.825739,-0.032778,-0.159362,...,-0.076677,47.269340,74.060886,-0.038431,-2.507999,-0.627833,0.0,0.0,AWI,417000.0
2020-05-18,-0.006166,1.537272,4.97,-0.027718,0.0,5.39,4.95,0.306075,0.068929,0.000768,...,0.197621,50.443179,5.390000,-0.084507,0.333793,0.015167,0.0,0.0,AUY,25037426.0


In [24]:
df.to_csv('df_409.csv', index=True)

In [25]:
df.columns

Index(['acc', 'adx', 'close', 'direction', 'extrema', 'high', 'low', 'macd',
       'macd_100_200_norm', 'macd_50_100_norm', 'macd_50_150_norm',
       'macd_50_200_norm', 'open', 'pct_exp_mean_1_10', 'pct_exp_mean_1_5',
       'pct_exp_std_1_10', 'pct_exp_std_1_5', 'pct_lag_1', 'pct_lag_10',
       'pct_lag_11', 'pct_lag_12', 'pct_lag_13', 'pct_lag_14', 'pct_lag_15',
       'pct_lag_2', 'pct_lag_20', 'pct_lag_25', 'pct_lag_3', 'pct_lag_30',
       'pct_lag_4', 'pct_lag_5', 'pct_lag_6', 'pct_lag_7', 'pct_lag_8',
       'pct_lag_9', 'pct_r_mean_10_10', 'pct_r_mean_10_15', 'pct_r_mean_10_20',
       'pct_r_mean_10_5', 'pct_r_mean_15_10', 'pct_r_mean_15_15',
       'pct_r_mean_15_20', 'pct_r_mean_15_5', 'pct_r_mean_1_10',
       'pct_r_mean_1_15', 'pct_r_mean_1_20', 'pct_r_mean_1_5',
       'pct_r_mean_5_10', 'pct_r_mean_5_15', 'pct_r_mean_5_20',
       'pct_r_mean_5_5', 'pct_r_std_10_10', 'pct_r_std_10_15',
       'pct_r_std_10_20', 'pct_r_std_10_5', 'pct_r_std_15_10',
       'pct_r_std_

In [26]:
df.describe()

,acc,adx,close,direction,extrema,high,low,macd,macd_100_200_norm,macd_50_100_norm,...,r_mean_50,r_mean_50_norm,rsi,sar,sar_norm,signal,slope,target_max,target_min,volume
count,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,...,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,947676.000000,9.476760e+05
mean,0.000004,1.076938,70.368105,0.002603,0.000349,71.116651,69.572381,0.226877,0.017256,0.007002,...,69.582396,0.003758,52.744185,70.002699,0.002334,0.224274,0.033757,0.032255,0.032604,3.280336e+06
std,0.202384,0.324353,156.113547,1.092909,0.254674,157.841330,154.280979,3.454114,0.073469,0.054772,...,154.115373,0.084673,16.372742,155.160920,0.064391,3.218069,1.212923,0.176676,0.177598,8.426007e+06
min,-27.284103,0.350869,0.630000,-176.042706,-1.000000,0.670000,0.600000,-422.071266,-1.177927,-0.877998,...,0.918400,-3.332323,2.054488,0.600000,-1.731492,-324.455425,-227.738333,0.000000,0.000000,0.000000e+00
25%,-0.019953,0.837105,20.650000,-0.109206,0.000000,20.890000,20.400000,-0.233723,-0.013260,-0.015618,...,20.516350,-0.027991,41.102094,20.554272,-0.034005,-0.214209,-0.092833,0.000000,0.000000,3.558000e+05
50%,-0.000221,1.024198,38.300000,0.003020,0.000000,38.700000,37.880000,0.103229,0.025186,0.012731,...,37.948500,0.013026,53.176024,38.132303,0.011494,0.101404,0.017000,0.000000,0.000000,1.054600e+06
75%,0.019441,1.263642,69.100000,0.117294,0.000000,69.800000,68.380000,0.544993,0.057769,0.037277,...,68.421650,0.047856,64.719697,68.851444,0.040357,0.522361,0.145500,0.000000,0.000000,2.881100e+06
max,32.292436,2.973434,4037.770000,81.022515,1.000000,4071.130000,3986.950000,128.593034,0.347827,0.290470,...,3880.178200,0.518308,98.879353,4071.130000,0.615729,112.338422,84.727333,1.000000,1.000000,6.166205e+08


### 4. Modelling.
### Prepare Features(X) and target local minima(y).

In [28]:
num_cols = ['pct_lag_1','pct_lag_2', 'pct_lag_3', 'pct_lag_4', 
             'pct_lag_5', 'pct_lag_6','pct_lag_7', 'pct_lag_8', 'pct_lag_9', 
             'pct_lag_10', 'pct_lag_11','pct_lag_12', 'pct_lag_13', 'pct_lag_14',
             'pct_lag_15', 'pct_lag_20', 'pct_lag_25', 'pct_lag_30',
             'pct_r_mean_1_5','pct_r_std_1_5','pct_r_mean_1_10', 'pct_r_std_1_10',
             'pct_r_mean_1_15', 'pct_r_std_1_15', 'pct_r_mean_1_20', 'pct_r_std_1_20',
             'pct_r_mean_5_5','pct_r_std_5_5', 'pct_r_mean_5_10', 'pct_r_std_5_10',
             'pct_r_mean_5_15','pct_r_std_5_15', 'pct_r_mean_5_20', 'pct_r_std_5_20',
             'pct_r_mean_10_5', 'pct_r_std_10_5', 'pct_r_mean_10_10','pct_r_std_10_10',
             'pct_r_mean_10_15', 'pct_r_std_10_15','pct_r_mean_10_20', 'pct_r_std_10_20', 
             'pct_r_mean_15_5','pct_r_std_15_5', 'pct_r_mean_15_10', 'pct_r_std_15_10',
             'pct_r_mean_15_15', 'pct_r_std_15_15', 'pct_r_mean_15_20','pct_r_std_15_20',
             'pct_exp_mean_1_5', 'pct_exp_std_1_5','pct_exp_mean_1_10', 'pct_exp_std_1_10', 
             'r_mean_50_norm','r_mean_100_norm','r_mean_150_norm','r_mean_200_norm',
             'macd_50_100_norm', 'macd_50_150_norm','macd_50_200_norm', 'macd_100_200_norm',
             'direction','slope', 'acc', 'sar_norm', 'adx', 'rsi'] 

cat_cols = ['ticker']

used_cols = num_cols + cat_cols

X = df[used_cols]
y = df['target_min']


### 4.2. Vanilla LogReg model.

In [31]:
### Make numerical pipe
imputer = SimpleImputer()
scaler = RobustScaler()
pca = PCA(n_components=0.95, random_state = 42)
num_pipe = make_pipeline(imputer,scaler,pca)

### Make categorical pipe
encoder = WOEEncoder()
cat_pipe = make_pipeline(encoder)

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[('num', num_pipe, num_cols),
                  ('cat', cat_pipe, cat_cols)])

# Define model
model = LogisticRegression(class_weight='balanced', random_state=42,C=10,solver = 'newton-cg',
                          fit_intercept=True,penalty='l2')

# Make main pipeline
pipe = make_pipeline(preprocessor, model)

# Get cross val score
scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc')

# Print results
print(f'ROC-AUC scores:     {scores}')
print('-------------------')
print(f'Mean ROC-AUC score: {scores.mean()} +/-{scores.std()}')
print('-------------------')


ROC-AUC scores:     [0.92535036 0.92385927 0.92738442 0.92391952 0.9205724 ]
-------------------
Mean ROC-AUC score: 0.9242171948587397 +/-0.0022278543850685175
-------------------


### 4.3. Score Card.

In [373]:
# Make numerical pipe
imputer = SimpleImputer()
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
scaler = RobustScaler()
encoder = WOEEncoder()
pca = PCA(n_components=0.90, random_state = 42)
num_pipe = make_pipeline(imputer, binner, scaler, encoder,pca)

# Make categorical pipe
encoder = WOEEncoder()
cat_pipe = make_pipeline(encoder)

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[('num', num_pipe, num_cols),
                  ('cat', cat_pipe, cat_cols)])

# Define model
model = LogisticRegression(class_weight='balanced', random_state=42,C=10,solver = 'lbfgs',
                          fit_intercept=True,penalty='l2')

# Make main pipeline
pipe = make_pipeline(preprocessor, model)

# Get cross val score
scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc')

# Print results
print(f'ROC-AUC scores:     {scores}')
print('-------------------')
print(f'Mean ROC-AUC score: {scores.mean()} +/-{scores.std()}')
print('-------------------')

ROC-AUC scores:     [0.93040966 0.92843141 0.93306465 0.93174712 0.93159175]
-------------------
Mean ROC-AUC score: 0.9310489163715777 +/-0.0015559782449752909
-------------------


### 4.4. Train Test split.

In [374]:
from sklearn.model_selection import train_test_split
X = df[used_cols]
y = df['target_min']

end = '2017-05-18'
start = '2017-05-19'

X_train, X_test = X[:end],X[start:] 
y_train, y_test= y[:end],y[start:]  


In [375]:
y_train.value_counts()

0.0    618374
1.0     20916
Name: target_min, dtype: int64

In [376]:
pipe.fit(X_train, y_train)

y_hat_train = pipe.predict(X_train)
y_hat_test = pipe.predict(X_test)

prob_train = pipe.predict_proba(X_train)[:,1]
prob_test = pipe.predict_proba(X_test)[:,1]

print(f'ROC-AUC train_scores: {roc_auc_score(y_train, prob_train)}')
print('---------------------')
print(f'ROC-AUC test_scores: {roc_auc_score(y_test, prob_test)}')
print('---------------------')


ROC-AUC train_scores: 0.9311790451759321
---------------------
ROC-AUC test_scores: 0.932023215406047
---------------------


In [377]:
thresh = 0.8
pred_train = (prob_train >= thresh).astype(np.int)
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

         0.0       0.99      0.91      0.95    618374
         1.0       0.21      0.70      0.32     20916

    accuracy                           0.90    639290
   macro avg       0.60      0.80      0.64    639290
weighted avg       0.96      0.90      0.93    639290



In [378]:
confusion_matrix(y_train,pred_train)

array([[563750,  54624],
       [  6368,  14548]])

In [379]:
thresh = 0.8
pred = (prob_test >= thresh).astype(np.int)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.99      0.91      0.95    298404
         1.0       0.21      0.70      0.32      9982

    accuracy                           0.90    308386
   macro avg       0.60      0.81      0.64    308386
weighted avg       0.96      0.90      0.93    308386



In [380]:
confusion_matrix(y_test,pred)

array([[271774,  26630],
       [  2965,   7017]])

#### Observation:
Here we see low precision, however, due to nature of the target(I used true local minima with 10 days to each side), model picks up some local minima with order of less than 10 days. As was shown in notebook with 3 stock tickers there not so many real false positives. Implementation of good stop-loss will be essential.

#### Save files:

In [381]:
end = '2017-05-18'
start = '2017-05-19'

df_trained = df[:end]
df_trained['predicted']=pred_train
#df_trained.to_csv('df_409_trained.csv', index=True)

df_tested = df[start:]
df_tested['predicted']=pred
#df_tested.to_csv('df_409_tested.csv', index=True)

### 5. Check ROI.

In [382]:
df_roi, df_concat = roi_calc(df_tested)

In [391]:
df_roi.sort_values(by='roi',ascending=False)

,roi
ticker,
DXCM,5.820245
PODD,4.210442
LULU,3.440147
DG,2.980803
CWST,2.958196
...,...
AA,0.260904
ALKS,0.237282
BKD,0.221539


In [395]:
### ROI for portfolio of 409 shares:
(df_roi.sum().values/len(df_roi))[0]

1.2480644062245474

In [396]:
df_roi.describe()

,roi
count,409.000000
mean,1.248064
std,0.596561
min,0.142075
25%,0.811553
50%,1.163492
75%,1.522759
max,5.820245


In [397]:
df_concat

,date,ticker,open,high,low,close,predicted,pos_in,pos_out,roi,pct_loss,roi_cumprod
0,2017-06-12,CORT,11.47,11.71,10.93,10.97,1,1,0,1.000000,0.000000,1.000000
1,2017-06-13,CORT,11.02,11.16,10.63,10.88,0,0,0,1.000000,-0.008204,1.000000
2,2017-06-14,CORT,10.89,10.90,10.49,10.55,1,1,1,0.961714,-0.030331,0.961714
3,2017-06-15,CORT,10.50,10.76,10.40,10.69,0,0,0,1.000000,0.013270,0.961714
4,2017-06-16,CORT,10.59,10.81,10.50,10.66,0,0,0,1.000000,-0.002806,0.961714
...,...,...,...,...,...,...,...,...,...,...,...,...
674,2020-05-12,SEIC,52.39,52.39,49.44,49.44,0,0,0,1.000000,-0.047582,0.808269
675,2020-05-13,SEIC,49.20,50.11,48.61,49.97,0,0,0,1.000000,0.010720,0.808269
676,2020-05-14,SEIC,49.20,50.80,48.70,50.64,0,0,0,1.000000,0.013408,0.808269
677,2020-05-15,SEIC,50.30,51.04,49.94,50.77,0,0,0,1.000000,0.002567,0.808269


In [398]:
long_inv = df_concat.groupby('ticker')['close'].agg(['first','last'])
long_inv['roi'] = long_inv['last']/long_inv['first']
long_inv

,first,last,roi
ticker,,,
AA,31.780,7.80,0.245437
AAPL,142.800,314.96,2.205602
ABEV,5.037,2.13,0.422871
ABMD,140.470,191.05,1.360077
ACGL,30.930,24.36,0.787585
...,...,...,...
WU,16.810,19.40,1.154075
XEL,42.830,60.13,1.403922
XPER,27.860,15.45,0.554559


In [401]:
# long investment ROI
(long_inv['roi'].sum())/len(long_inv)

1.3274495639257442

### conclusion:
1) Using big number of shares as portfolio with frequent trades not very efficient,when comapared to long investment. Model ROI 1.248 vs long_term ROI 1.327.

2) Good strategy of selection is required

3) However, as shown in SP500 notebook, model shows significantly higher ROI over long term ROI, when used with lower number of stocks (or better selection of stocks) and with good stop-loss strategy.